<a href="https://colab.research.google.com/github/parag-ghungrudkar/Coursera_Capstone/blob/main/Coursera_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM Data Science Capstone Project
###### The Battle of neighborhood: The business object of this project is to cluster neighborhood with similar venues together. Further explanation will be detailed in this ...placeholder

###Part 1:
Creating a notebook for the project. I am using google colab for the purpose.

In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np

In [ ]:
#printing the required string
print('Hello capstone project coursera')

## Part 2:

1. Created pandas dataframe for the neighborhood in Toronto using web scrapping
2. Found nearby venues for each neighborhood using FourSquare API
3. One hot encoding for categories
4. Clustering analysis of the neighborhood

In [1]:
#importing required libraries
import requests
from bs4 import BeautifulSoup

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(url).text

In [4]:
soup = BeautifulSoup(html)

In [5]:
tables = soup.find_all('table')

In [6]:
tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B